In [47]:
#importing the Libraies
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

In [48]:
# Reading the Dataset
dataset = pd.read_csv('Social_Network_Ads.csv')

In [49]:
dataset

,User ID,Gender,Age,EstimatedSalary,Purchased
0,15624510,Male,19,19000,0
1,15810944,Male,35,20000,0
2,15668575,Female,26,43000,0
3,15603246,Female,27,57000,0
4,15804002,Male,19,76000,0
...,...,...,...,...,...
395,15691863,Female,46,41000,1
396,15706071,Male,51,23000,1
397,15654296,Female,50,20000,1
398,15755018,Male,36,33000,0


In [51]:
dataset=pd.get_dummies(dataset, dtype=int, drop_first=True)

In [52]:
dataset

,User ID,Age,EstimatedSalary,Purchased,Gender_Male
0,15624510,19,19000,0,1
1,15810944,35,20000,0,1
2,15668575,26,43000,0,0
3,15603246,27,57000,0,0
4,15804002,19,76000,0,1
...,...,...,...,...,...
395,15691863,46,41000,1,0
396,15706071,51,23000,1,1
397,15654296,50,20000,1,0
398,15755018,36,33000,0,1


In [53]:
independent=dataset[['Age', 'EstimatedSalary','Gender_Male']]
dependent=dataset['Purchased']

In [54]:
#split into training set and test
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(independent, dependent, test_size = 1/3, random_state = 0)


In [55]:
from sklearn.preprocessing import StandardScaler
StandardScaler = StandardScaler()
X_train = StandardScaler.fit_transform(X_train)
X_test = StandardScaler.transform(X_test)

In [56]:
from sklearn.tree import DecisionTreeClassifier

In [57]:
#https://scikit-learn.org/stable/modules/model_evaluation.html#scoring-parameter

In [58]:
from sklearn.model_selection import GridSearchCV

param_grid = {'criterion':['gini','entropy'],
              'max_features': ['auto','sqrt','log2'],
              'splitter':['best','random']} 



grid = GridSearchCV(DecisionTreeClassifier(), param_grid, refit = True, verbose = 3,n_jobs=-1,scoring='f1_weighted') 
   
# fitting the model for grid search 
grid.fit(X_train, y_train) 
 



Fitting 5 folds for each of 12 candidates, totalling 60 fits
[CV 4/5] END criterion=gini, max_features=auto, splitter=best;, score=nan total time=   0.0s
[CV 1/5] END criterion=gini, max_features=auto, splitter=best;, score=nan total time=   0.0s
[CV 5/5] END criterion=gini, max_features=auto, splitter=best;, score=nan total time=   0.0s
[CV 1/5] END criterion=gini, max_features=auto, splitter=random;, score=nan total time=   0.0s
[CV 2/5] END criterion=gini, max_features=auto, splitter=random;, score=nan total time=   0.0s
[CV 3/5] END criterion=gini, max_features=auto, splitter=random;, score=nan total time=   0.0s
[CV 4/5] END criterion=gini, max_features=auto, splitter=random;, score=nan total time=   0.0s
[CV 5/5] END criterion=gini, max_features=auto, splitter=random;, score=nan total time=   0.0s
[CV 1/5] END criterion=gini, max_features=sqrt, splitter=best;, score=0.788 total time=   0.0s
[CV 2/5] END criterion=gini, max_features=sqrt, splitter=best;, score=0.887 total time=   

/home/deehub/JoinDeeHub/.venv/lib/python3.12/site-packages/sklearn/model_selection/_validation.py:528: FitFailedWarning: 
20 fits failed out of a total of 60.
The score on these train-test partitions for these parameters will be set to nan.
If these failures are not expected, you can try to debug them by setting error_score='raise'.

Below are more details about the failures:
--------------------------------------------------------------------------------
8 fits failed with the following error:
Traceback (most recent call last):
  File "/home/deehub/JoinDeeHub/.venv/lib/python3.12/site-packages/sklearn/model_selection/_validation.py", line 866, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
  File "/home/deehub/JoinDeeHub/.venv/lib/python3.12/site-packages/sklearn/base.py", line 1382, in wrapper
    estimator._validate_params()
  File "/home/deehub/JoinDeeHub/.venv/lib/python3.12/site-packages/sklearn/base.py", line 436, in _validate_params
    validate_parameter_c

GridSearchCV(estimator=DecisionTreeClassifier(), n_jobs=-1,
             param_grid={'criterion': ['gini', 'entropy'],
                         'max_features': ['auto', 'sqrt', 'log2'],
                         'splitter': ['best', 'random']},
             scoring='f1_weighted', verbose=3)

In [59]:
# print best parameter after tuning 
#print(grid.best_params_) 
result = grid.cv_results_
#print(result)
grid_predictions = grid.predict(X_test) 
   

from sklearn.metrics import confusion_matrix
confusion_matrix = confusion_matrix(y_test, grid_predictions)



# print classification report 
from sklearn.metrics import classification_report
classification_report = classification_report(y_test, grid_predictions)




In [60]:

from sklearn.metrics import f1_score
f1_macro=f1_score(y_test,grid_predictions,average='weighted')
print("The f1_macro value for best parameter {}:".format(grid.best_params_),f1_macro)


The f1_macro value for best parameter {'criterion': 'entropy', 'max_features': 'log2', 'splitter': 'best'}: 0.9180821729616219


In [61]:
print("The confusion Matrix:\n",confusion_matrix)

The confusion Matrix:
 [[79  6]
 [ 5 44]]


In [62]:
print("The report:\n",classification_report)

The report:
               precision    recall  f1-score   support

           0       0.94      0.93      0.93        85
           1       0.88      0.90      0.89        49

    accuracy                           0.92       134
   macro avg       0.91      0.91      0.91       134
weighted avg       0.92      0.92      0.92       134



In [63]:
from sklearn.metrics import roc_auc_score

roc_auc_score(y_test,grid.predict_proba(X_test)[:,1])


np.float64(0.9136854741896759)

In [64]:
table=pd.DataFrame.from_dict(result)

In [65]:
table

,mean_fit_time,std_fit_time,mean_score_time,std_score_time,param_criterion,param_max_features,param_splitter,params,split0_test_score,split1_test_score,split2_test_score,split3_test_score,split4_test_score,mean_test_score,std_test_score,rank_test_score
0,0.002061,0.000805,0.000000,0.000000,gini,auto,best,"{'criterion': 'gini', 'max_features': 'auto', ...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,9
1,0.001103,0.000178,0.000000,0.000000,gini,auto,random,"{'criterion': 'gini', 'max_features': 'auto', ...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,9
2,0.004106,0.001243,0.009039,0.003517,gini,sqrt,best,"{'criterion': 'gini', 'max_features': 'sqrt', ...",0.787654,0.886792,0.833323,0.925272,0.844370,0.855482,0.047045,4
3,0.003800,0.001603,0.008695,0.003483,gini,sqrt,random,"{'criterion': 'gini', 'max_features': 'sqrt', ...",0.869532,0.797126,0.756031,0.795256,0.833653,0.810320,0.038467,7
4,0.004366,0.000684,0.007920,0.001927,gini,log2,best,"{'criterion': 'gini', 'max_features': 'log2', ...",0.867478,0.813511,0.833323,0.905069,0.886792,0.861235,0.033676,2
5,0.003930,0.001894,0.006432,0.002406,gini,log2,random,"{'criterion': 'gini', 'max_features': 'log2', ...",0.847141,0.714931,0.683578,0.888646,0.925272,0.811914,0.095765,6
6,0.001431,0.001166,0.000000,0.000000,entropy,auto,best,"{'criterion': 'entropy', 'max_features': 'auto...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,9
7,0.001352,0.001233,0.000000,0.000000,entropy,auto,random,"{'criterion': 'entropy', 'max_features': 'auto...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,9
8,0.003229,0.001866,0.005318,0.002560,entropy,sqrt,best,"{'criterion': 'entropy', 'max_features': 'sqrt...",0.867478,0.828959,0.832483,0.832483,0.923510,0.856982,0.036116,3
9,0.002444,0.000944,0.006612,0.003288,entropy,sqrt,random,"{'criterion': 'entropy', 'max_features': 'sqrt...",0.804764,0.828959,0.796284,0.753180,0.826499,0.801937,0.027389,8
